## 3.1 Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression 

import time

## 3.2 Load The  Forest Cover Type Data

In [2]:
covtype = pd.read_csv('../data/covtype_step2_features.csv')
covtype.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,interact_Hillshade_9am3pm,interact_Hillshade_9amNoon,interact_Hillshade_3pmNoon,Euclidean_Distance_To_Hydrology
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,5,32708,51272,34336,258.000000
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,5,33220,51700,35485,212.084889
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,2,31590,55692,32130,275.769832
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,2,29036,56644,29036,269.235956
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,5,33000,51480,35100,153.003268


In [3]:
X = covtype.loc[:, covtype.columns != 'Cover_Type']
y = covtype.loc[:, covtype.columns == 'Cover_Type']

## Add Features 

## 3.3 Train/Test Split

In [4]:
#split data into trainning set, validation set, and test set
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state = 42, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state  = 42, stratify = y_train)

In [6]:
X_val.shape

(92962, 58)

## 3.4 Target Encoding On Categorical variable

Our data set has categorical features that are one-hot encoded,where soil type consists of 40 types and wilderness area consists of 4 types. The higher dimension of the feature matrix sometimes may cause the algorithm having a hard time to learn form the data, so we also want to explore if target encoding the categorical features will results in a better outcome.  

In [139]:
def onehot_to_label(X, cat_name):   
    '''transform the one hot encoding columns to a label column'''
    
    X = X.iloc[:, X.columns.str.contains(pat = cat_name + '.*')] 
    Label = X.apply(lambda row : row.argmax(), axis = 1)
    return Label

In [140]:
def target_encoding(X, target, cat_name):
    '''return the posterior probility of a cover type given a sample belonging to a certain category'''
    
    label = onehot_to_label(X, cat_name)
    label_target = pd.DataFrame({cat_name: label, 'target': target.Cover_Type})
    posterior_prob = pd.DataFrame(index = range(label.nunique()))
    
    #total 7 cover types
    n = 7
    for i in range(1, n+1):
        label_target['ith_Covtype'] = np.array(target == i).astype('int')
        encoded_feature = 'PProb_CovType' + str(i) + '|' + cat_name
        posterior_prob[encoded_feature] = label_target[['ith_Covtype', cat_name]
                                                      ].groupby(cat_name).mean() 

        label_target.drop(columns  = 'ith_Covtype', inplace = True)  
        
    return posterior_prob
 

In [141]:
def merge_encoded_features(X, encoded_features, cat_name):
    '''replace the one-hot encoded features by target encoded features'''
    X_copy = X.copy()
    X_copy[cat_name] = onehot_to_label(X_copy, cat_name)
    new_X = X_copy.merge(encoded_features, how = 'left', left_on = cat_name, right_index = True
                        ).drop(columns = X_copy.columns[X_copy.columns.str.contains('^' + cat_name)])
    return new_X
    

### 3.4.1 Target Encoding On Soil Type

In [142]:
#each cell represents the posterior probability of a cover type given the soil type,eg:encoded_SoilType.iloc[0,1] is 
#the posterior probability of a sample belonging to Cover type 1 given it belonging to soil type 
encoded_SoilType = target_encoding(X_train, y_train, 'Soil_Type')
encoded_SoilType.head()

,PProb_CovType1|Soil_Type,PProb_CovType2|Soil_Type,PProb_CovType3|Soil_Type,PProb_CovType4|Soil_Type,PProb_CovType5|Soil_Type,PProb_CovType6|Soil_Type,PProb_CovType7|Soil_Type
0,0.00000,0.000000,0.690814,0.059318,0.000000,0.249869,0.000000
1,0.00000,0.112611,0.660270,0.014973,0.034795,0.177351,0.000000
2,0.00000,0.248967,0.493482,0.213355,0.000000,0.044197,0.000000
3,0.01448,0.264039,0.605137,0.012843,0.047973,0.049232,0.006296
4,0.00000,0.000000,0.611969,0.033784,0.000000,0.354247,0.000000


In [143]:
#add the target encoded soil type feature to the feature matrix
#for validation set and test set, we transfer the categorical features using the benchmarks from the tranning set,
#in order to avoid data leakage
X_train_target = merge_encoded_features(X_train, encoded_SoilType, 'Soil_Type')
X_val_target = merge_encoded_features(X_val, encoded_SoilType, 'Soil_Type')
X_test_target = merge_encoded_features(X_test, encoded_SoilType, 'Soil_Type')

### 3.4.2 Target Encoding On Wilderness Area

In [144]:
#each cell represents the posterior probability of a cover type given the wilderness area,
#eg:encoded_WildernessArea.iloc[0,1] is the posterior probability of a sample belonging to Cover type 1 given it is
#in wilderness area 0.
encoded_WildernessArea = target_encoding(X_train, y_train, 'Wilderness_Area')
encoded_WildernessArea

,PProb_CovType1|Wilderness_Area,PProb_CovType2|Wilderness_Area,PProb_CovType3|Wilderness_Area,PProb_CovType4|Wilderness_Area,PProb_CovType5|Wilderness_Area,PProb_CovType6|Wilderness_Area,PProb_CovType7|Wilderness_Area
0,0.405578,0.560433,0.000000,0.000000,0.014547,0.000000,0.019442
1,0.620827,0.301871,0.000000,0.000000,0.000000,0.000000,0.077302
2,0.345674,0.494063,0.056129,0.000000,0.022481,0.029854,0.051799
3,0.000000,0.080725,0.580725,0.074188,0.000000,0.264361,0.000000


In [145]:
#add the target encoded wilderness area feature to the feature matrix
X_train_target = merge_encoded_features(X_train_target, encoded_WildernessArea, 'Wilderness_Area')
X_val_target = merge_encoded_features(X_val_target, encoded_WildernessArea, 'Wilderness_Area')
X_test_target = merge_encoded_features(X_test_target, encoded_WildernessArea, 'Wilderness_Area')

In [146]:
X_train_target.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'interact_Hillshade_9am3pm',
       'interact_Hillshade_9amNoon', 'interact_Hillshade_3pmNoon',
       'Euclidean_Distance_To_Hydrology', 'PProb_CovType1|Soil_Type',
       'PProb_CovType2|Soil_Type', 'PProb_CovType3|Soil_Type',
       'PProb_CovType4|Soil_Type', 'PProb_CovType5|Soil_Type',
       'PProb_CovType6|Soil_Type', 'PProb_CovType7|Soil_Type',
       'PProb_CovType1|Wilderness_Area', 'PProb_CovType2|Wilderness_Area',
       'PProb_CovType3|Wilderness_Area', 'PProb_CovType4|Wilderness_Area',
       'PProb_CovType5|Wilderness_Area', 'PProb_CovType6|Wilderness_Area',
       'PProb_CovType7|Wilderness_Area'],
      dtype='object')

## 3.5 Scale The Data

### 3.5.1 Scale Data With One Hot Encoded Features

In [147]:
#extract categorical indice because we dont want to scale one hot encoded features
cat_index = X.columns.str.contains('Wilderness_Area|Soil_Type', regex=True)  

In [148]:
#standardize the training set
cat_features = X_train.loc[:, cat_index]
noncat_features = X_train.loc[:, ~cat_index]

scaler = StandardScaler()
scaler.fit(noncat_features)
scaled_noncat_features = scaler.transform(noncat_features)
#concatenate the scaled numeric features and categorical features
X_train_onehot = pd.concat([pd.DataFrame(scaled_noncat_features,
                                         columns = X.columns[~cat_index],
                                         index = cat_features.index
                                        ),cat_features], axis = 1)


#standardize validation set
cat_features = X_val.loc[:, cat_index]
noncat_features = X_val.loc[:, ~cat_index]
scaled_noncat_features = scaler.transform(noncat_features)
#concatenate the scaled numeric features and categorical features
X_val_onehot = pd.concat([pd.DataFrame(scaled_noncat_features,
                                       columns = X.columns[~cat_index],
                                       index = cat_features.index
                                      ), cat_features], axis = 1)


#standardize test set
cat_features = X_test.loc[:, cat_index]
noncat_features = X_test.loc[:, ~cat_index]
scaled_noncat_features = scaler.transform(noncat_features)
#concatenate the scaled numeric features and categorical features
X_test_onehot = pd.concat([pd.DataFrame(scaled_noncat_features,
                                        columns = X.columns[~cat_index],
                                        index = cat_features.index
                                       ), cat_features], axis = 1)

### 3.5.2 Scale Data With Target Encoded Features

In [149]:
#store column names for data storage in the future
col_names_target = X_train_target.columns

#standardize trianning set
scaler = StandardScaler()
scaler.fit(X_train_target)
X_train_target = scaler.transform(X_train_target)

#standardize validation set
X_val_target = scaler.transform(X_val_target)

#standardize validation set
X_test_target = scaler.transform(X_test_target)

## 3.6 Baseline Model

Next, we use logistic regression to fit our baseline model. 

In [78]:
start = time.process_time()

#fit the baseline model
baseline = LogisticRegression(max_iter = 3000,random_state = 42).fit(X_train_onehot, y_train.values.ravel())
pred_y_train = baseline.predict(X_train_onehot)

end = time.process_time()
print('The processing time for the baseline classifier is: ' + str(end-start) + ' seconds.')

The processing time for the baseline classifier is: 446.51308199999994 seconds.


In [79]:
print(classification_report(y_train.values.ravel(),pred_y_train))

              precision    recall  f1-score   support

           1       0.71      0.70      0.71    135578
           2       0.75      0.80      0.77    181312
           3       0.69      0.79      0.74     22882
           4       0.66      0.45      0.54      1759
           5       0.48      0.03      0.06      6075
           6       0.49      0.30      0.37     11115
           7       0.74      0.58      0.65     13126

    accuracy                           0.73    371847
   macro avg       0.65      0.52      0.55    371847
weighted avg       0.72      0.73      0.72    371847



In [80]:
# predict the validation set
pred_y_val = baseline.predict(X_val_onehot)
print(classification_report(y_val.values.ravel(),pred_y_val))

              precision    recall  f1-score   support

           1       0.71      0.70      0.70     33894
           2       0.75      0.80      0.77     45328
           3       0.70      0.80      0.75      5721
           4       0.63      0.43      0.51       439
           5       0.47      0.03      0.06      1519
           6       0.49      0.31      0.38      2779
           7       0.75      0.58      0.66      3282

    accuracy                           0.73     92962
   macro avg       0.64      0.52      0.55     92962
weighted avg       0.72      0.73      0.72     92962



The classification report shows that the baseline model performs poorly in both of the trainning set and validation set. The scores on training set and validation have no big difference, indicating that the baseline model is underfitting and fails to captures some important information contributing to our target value.

The macro average f1-scores on training set and validation set are only 0.55 and 0.54, which is mainly becasue the model is unable to sperate the minority classes from the majority one. We can see that only 4% of the samples belonging cover type 5 are successfully classfied. Less than 50% of the samples belonging to covertype 4 and 6 are successfully classfied.

## 3.7 Compare One Hot Encoding And Target Encoding

As mentioned above,one-hot encoding makes our data sparser at some levels and increases the computational cost. Now, holding the hyperparameters the same, let's explore if target encoding outperforms one-hot encoding.

In [106]:
start = time.process_time()

#fit data with target encoded features
clf1 = LogisticRegression(max_iter = 3000,random_state = 42).fit(X_train_target,y_train.values.ravel())
pred_y_train = clf1.predict(X_train_target)

end = time.process_time()
print('The processing time for the classifier clf1 is: ' + str(end-start) + ' seconds.')

The processing time for the classifier clf1 is: 382.86451199999965 seconds.


In [107]:
print(classification_report(y_train.values.ravel(),pred_y_train))

              precision    recall  f1-score   support

           1       0.71      0.70      0.70    135578
           2       0.75      0.80      0.77    181312
           3       0.69      0.79      0.74     22882
           4       0.65      0.46      0.54      1759
           5       0.48      0.04      0.07      6075
           6       0.48      0.29      0.36     11115
           7       0.73      0.56      0.63     13126

    accuracy                           0.72    371847
   macro avg       0.64      0.52      0.54    371847
weighted avg       0.72      0.72      0.72    371847



In [108]:
#fit the baseline model
pred_y_val = clf1.predict(X_val_target)
print(classification_report(y_val.values.ravel(),pred_y_val))

              precision    recall  f1-score   support

           1       0.71      0.69      0.70     33894
           2       0.75      0.80      0.77     45328
           3       0.69      0.80      0.74      5721
           4       0.60      0.43      0.50       439
           5       0.48      0.04      0.07      1519
           6       0.48      0.28      0.36      2779
           7       0.73      0.57      0.64      3282

    accuracy                           0.72     92962
   macro avg       0.63      0.52      0.54     92962
weighted avg       0.72      0.72      0.71     92962



The processing time for this model is slightly less than the baseline, which meets our expectation since target encoding has reduced the total number of features from 58 to 28, more than a half! 

The classificaiton reports shows that the socres between two models are quite close. In order words, 28 features are likely to include majority of the information in 50 features. Thus, we will choose 28 features in our fulture modeling, considering less computational cost and mermory usage.

## 3.8 Save Data

In [154]:
#save trainning set
train_new = pd.DataFrame(X_train_target, columns = col_names_target,index = y_train.index).join(y_train)
datapath = '../data'
train_new.to_csv(datapath + '/training_step3.csv', index = False)

#save validation set
val_new = pd.DataFrame(X_val_target, columns = col_names_target, index = y_val.index).join(y_val)
val_new.to_csv(datapath + '/validation_step3.csv', index = False)

#save validation set
test_new = pd.DataFrame(X_test_target, columns = col_names_target, index = y_test.index).join(y_test)
test_new.to_csv(datapath + '/test_step3.csv', index= False)
